In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import itertools
import collections

In [2]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1560,tt0304141,130000000,789804554,Harry Potter and the Prisoner of Azkaban,Daniel Radcliffe|Rupert Grint|Emma Watson|Gary...,Alfonso CuarÃ³n,Something wicked this way comes.,"Harry, Ron and Hermione return to Hogwarts for...",141,Adventure|Fantasy|Family,1492 Pictures|Warner Bros.|Heyday Films|P of A...,5/31/2004,7.4,2004
472,tt1212419,50000000,105197635,Hereafter,Matt Damon|Bryce Dallas Howard|George McLaren|...,Clint Eastwood,Touched by death. Changed by life.,A supernatural thriller centered on three peop...,129,Drama|Fantasy,Amblin Entertainment|Malpaso Productions|Warne...,10/15/2010,5.5,2010
1764,tt0463998,21000000,41170784,Freedom Writers,Hilary Swank|Scott Glenn|Imelda Staunton|Patri...,Richard LaGravenese,Their story. Their world. Their future.,A young teacher inspires her class of at-risk ...,123,Crime|Drama,Double Feature Films|Jersey Films|Kernos Filmp...,1/5/2007,7.5,2007
1887,tt0162983,40000000,36037909,Hanging Up,Meg Ryan|Diane Keaton|Lisa Kudrow|Walter Matth...,Diane Keaton,Every family has a few hang-ups.,A trio of sisters bond over their ambivalence ...,94,Comedy|Drama,Laurence Mark Productions|Columbia Pictures Co...,2/16/2000,5.2,2000
93,tt2268016,14800000,122413057,Magic Mike XXL,Channing Tatum|Matt Bomer|Joe Manganiello|Kevi...,Gregory Jacobs,Back to the grind.,Three years after Mike bowed out of the stripp...,115,Comedy|Drama|Music,Warner Bros.|Iron Horse Entertainment (II),7/1/2015,6.3,2015


In [3]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [4]:
answers = {} # создадим словарь для ответов

# тут другие ваши предобработки колонок например:

#the time given in the dataset is in string format.
#So we need to change this in datetime format
# ...

def get_season(month):
#функция определяет сезон по номеру месяца в году

    month = int(month)
    
    if month in (3,4,5):
        return 'spring'
    elif month in (6,7,8):
        return 'summer'
    elif month in (9,10,11):
        return 'autumn'
    elif month in (12,1,2):
        return 'winter'
    else:
        return '#error'

    
#преобразуем столбец с датой в тип "дата"
data.release_date = pd.to_datetime(data.release_date)

#добавляем столбцы "прибыль", "месяц релиза", "сезон релиза"
data['profit'] = data['revenue'] - data['budget']
data['release_month'] = data.release_date.apply(lambda x: x.month)
data['release_season'] = data.release_month.apply(get_season)

data.head(3)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,release_month,release_season
0,tt0369610,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,The park is open.,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,2015-06-09,6.5,2015,1363528810,6,summer
1,tt1392190,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,What a Lovely Day.,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,2015-05-13,7.1,2015,228436354,5,spring
2,tt2908446,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,Robert Schwentke,One Choice Can Destroy You,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,2015-03-18,6.3,2015,185238201,3,spring


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   imdb_id               1889 non-null   object        
 1   budget                1889 non-null   int64         
 2   revenue               1889 non-null   int64         
 3   original_title        1889 non-null   object        
 4   cast                  1889 non-null   object        
 5   director              1889 non-null   object        
 6   tagline               1889 non-null   object        
 7   overview              1889 non-null   object        
 8   runtime               1889 non-null   int64         
 9   genres                1889 non-null   object        
 10  production_companies  1889 non-null   object        
 11  release_date          1889 non-null   datetime64[ns]
 12  vote_average          1889 non-null   float64       
 13  release_year      

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [6]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
answers['1'] = 'Pirates of the Caribbean: On Stranger Tides (tt1298650)'
# если ответили верно, можете добавить комментарий со значком "+"
# +

In [7]:
# тут пишем ваш код для решения данного вопроса:
data[data.budget == data.budget.max()][['original_title', 'imdb_id']]

,original_title,imdb_id
723,Pirates of the Caribbean: On Stranger Tides,tt1298650


ВАРИАНТ 2

In [8]:
# можно добавлять разные варианты решения
data.sort_values('budget', ascending = False).head(1)[['original_title', 'imdb_id']]

,original_title,imdb_id
723,Pirates of the Caribbean: On Stranger Tides,tt1298650


# 2. Какой из фильмов самый длительный (в минутах)?

In [9]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
answers['2'] = data[data.runtime == data.runtime.max()].original_title.values[0]
answers['2']

'Gods and Generals'

In [10]:
#решение2

answers['2'] = data.sort_values('runtime',ascending = False).head(1).original_title.values[0]
answers['2']

'Gods and Generals'

# 3. Какой из фильмов самый короткий (в минутах)?





In [11]:
#решение 1

answers['3'] = data.sort_values('runtime').head(1).original_title.values[0]
answers['3']

'Winnie the Pooh'

In [12]:
#решение 2

answers['3'] = data[data.runtime == data.runtime.min()].original_title.values[0]
answers['3']

'Winnie the Pooh'

# 4. Какова средняя длительность фильмов?


In [13]:
answers['4'] = data.runtime.mean()
answers['4']

109.6585494970884

# 5. Каково медианное значение длительности фильмов? 

In [14]:
answers['5'] = data.runtime.median()
answers['5']

107.0

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [15]:
# лучше код получения столбца profit вынести в Предобработку что в начале

In [16]:
#решение1
answers['6'] = data.sort_values('profit', ascending = False).head(1).iloc[0,[3,0]].tolist()
answers['6']

['Avatar', 'tt0499549']

In [17]:
#решение2
answers['6'] = data[data.profit == data.profit.max()].iloc[0,[3,0]].tolist()
answers['6']

['Avatar', 'tt0499549']

# 7. Какой фильм самый убыточный? 

In [18]:
#решение1
answers['7'] = data.sort_values('profit').head(1).iloc[0,[3,0]].tolist()
answers['7']

['The Lone Ranger', 'tt1210819']

In [19]:
#решение2
answers['7'] = data[data.profit == data.profit.min()].iloc[0,[3,0]].tolist()
answers['7']

['The Lone Ranger', 'tt1210819']

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [20]:
#решение1
answers['8'] = data[data.revenue > data.budget].imdb_id.count()
answers['8']

1478

In [21]:
#решение2
answers['8'] = data[data.profit > 0].imdb_id.nunique()
answers['8']

1478

In [22]:
#решение3
answers['8'] = len(data[data.profit > 0])
answers['8']

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [23]:
#решение1

#выберем фильмы 2008 года
filtered_data = data[data.release_year == 2008]

#вычислим фильм с максимальной прибылью 2008 года
answers['9'] = filtered_data[filtered_data.profit == filtered_data.profit.max()].iloc[0,[3,0]].tolist()
answers['9']

['The Dark Knight', 'tt0468569']

In [24]:
#решение2

#отсортируем фильмы 2008 года по убыванию прибыли
sorted_data = data.query('release_year == 2008').sort_values('profit', ascending = False)

#вычислим первый фильм
answers['9'] = sorted_data.head(1)[['original_title', 'imdb_id']].values[0,[0,1]].tolist()
answers['9']

['The Dark Knight', 'tt0468569']

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [25]:
#решение1
#определим фильмы за период с 2012 по 2014 год
filtered_data = data[data.release_year.between(2012,2014)]

#вычислим фильм с минимальной прибылью
answers['10'] = filtered_data[filtered_data.profit == filtered_data.profit.min()].iloc[0,[3,0]].tolist()
answers['10']

['The Lone Ranger', 'tt1210819']

In [26]:
#решение2

#определим фильмы за период с 2012 по 2014 год
filtered_data = data[data.release_year.between(2012,2014)]

#отсортируем фильмы по увеличению прибыли и вычислим первый фильм
answers['10'] = filtered_data.sort_values('profit').head(1).values[0,[3,0]].tolist()
answers['10']

['The Lone Ranger', 'tt1210819']

# 11. Какого жанра фильмов больше всего?

In [27]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале

ВАРИАНТ 2

In [28]:
#сначала группируем по "комбинированным" жанрам
#это сократит количество строк для дальнейшей обработки  
grouped_data = data.groupby('genres')['imdb_id'].count() 

#в цикле проходим полученный датафрейм. Считаем количество фильмов по каждому жанру отдельно
count_genres = {}
for key,value in grouped_data.items():
    
    genres = key.split('|')
    
    for genre in genres:
        
        if genre in count_genres:
            count_genres[genre] += value           
        else:
            count_genres.setdefault(genre, value)

#вычислим жанр с максимальным количеством фильмов в серии, созданной из словаря           
answers['11'] = pd.Series(count_genres).sort_values(ascending = False).head(1).index[0]
answers['11']

'Drama'

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [29]:
#сначала группируем по "комбинированным" жанрам фильмы, с прибылью больше нуля 
#это сократит количество строк для дальнейшей обработки  
grouped_data = data[data.profit>0].groupby('genres')['imdb_id'].count() 

#в цикле проходим полученный датафрейм. Считаем количество фильмов по каждому жанру отдельно
count_genres = collections.Counter()

for key, value in grouped_data.items():
    
    for genre in key.split('|'):
        
        count_genres[genre] += value
            
#вычислим жанр с максимальным количеством фильмов             
answers['12'] = pd.Series(count_genres).sort_values(ascending = False).head(1).index[0]
answers['12']

'Drama'

# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [30]:
answers['13'] = data.groupby('director')['revenue'].sum().sort_values(ascending = False).head(1).index[0]
answers['13']

'Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [31]:
#сначала группируем фильмы с жанром Action "комбинированным" режиссерам
#это сократит количество строк для дальнейшей обработки  
grouped_data = data[data.genres.str.contains('Action')]['director'].value_counts() 

#в цикле проходим полученную серию. Считаем количество фильмов по каждому режиссеру отдельно и заполненяем словарь 
count_movies = collections.Counter()

for key in grouped_data.keys():
    
    for director in key.split('|'):
        count_movies[director] += value           

#вычислим режиссера с максимальным количеством фильмов           
answers['14'] = pd.Series(count_movies).sort_values(ascending = False).head(1).index[0]
answers['14']

'Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [32]:
#сначала фильтруем группируем фильмы 2012 года по "комбинированным" актерам с подсчетом суммы кассовых сборов
#это сократит количество строк для дальнейшей обработки  
grouped_data = data[data.release_year == 2012].groupby('cast')['revenue'].sum()

#в цикле проходим полученный датафрейм. Считаем суммсу сборов по каждому актеру отдельно и заполненяем словарь 
count_movies = {}

for key, value in grouped_data.items():
    
    actors = key.split('|')

    for actor in actors:
        if actor in count_movies:
            count_movies[actor] += value            
        else:
            count_movies.setdefault(actor, value)  

#вычислим актера с максимальной суммой кассовых сборов в серии, созданной из словаря             
answers['15'] = pd.Series(count_movies).sort_values(ascending = False).head(1).index[0]
answers['15']

'Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [33]:
#сначала группируем фильмы с бюджетом выше среднего по "комбинированным" актерам 
#это сократит количество строк для дальнейшей обработки 
grouped_data = data[data.budget > data.budget.mean()].groupby('cast')['imdb_id'].count()

#в цикле проходим полученный датафрейм. Считаем количество фильмов по каждому актеру отдельно и заполненяем словарь
count_movies = {}

for key, value in grouped_data.items():
    
    actors = key.split('|')

    for actor in actors:
        
        if actor in count_movies:
            count_movies[actor] += value            
        else:
            count_movies.setdefault(actor, value)  

#вычислим актера с максимальным количеством фильмов в серии, созданной из словаря   
answers['16'] = pd.Series(count_movies).sort_values(ascending = False).head(1).index[0]
answers['16']

'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [34]:
#сначала группируем фильмы с Nicolas Cage по "комбинированным" жанрам 
#это сократит количество строк для дальнейшей обработки 
grouped_data = data[data.cast.str.contains('Nicolas Cage')]['genres'].value_counts()

#в цикле проходим полученную серию. Считаем количество фильмов по каждому жанру отдельно и заполненяем словарь
count_movies = collections.Counter()

for key in grouped_data.keys():

    for genre in key.split('|'):
        count_movies[genre] += value
            
#вычисляем жанр с максимальным количеством фильмов
answers['17'] = pd.Series(count_movies).sort_values(ascending = False).head(1).index[0]
answers['17']

'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [35]:
#отсортируем по убыванию прибыли фильмы, содержащие "Paramount Pictures" в названии студии
sorted_data = data[data.production_companies.str.contains('Paramount Pictures')].sort_values('profit')

#вычислим первый фильм в списке
answers['18'] = sorted_data.iloc[0][['original_title','imdb_id']].tolist()
answers['18']

['K-19: The Widowmaker', 'tt0267626']

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [36]:
answers['19'] = data.groupby('release_year').sum().sort_values('revenue',ascending = False).head(1).index[0]
answers['19']

2015

# 20. Какой самый прибыльный год для студии Warner Bros?

In [37]:
#сгруппируем фильмы, содержащие "Warner Bros" в названиях студий, и посчитаем прибыль по годам
grouped_data = data[data.production_companies.str.contains('Warner Bros')].groupby('release_year').sum()

#вычислим год с максимальной прибылью
answers['20'] = grouped_data.sort_values('profit',ascending = False).head(1).index[0]
answers['20']

2014

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [38]:
answers['21'] = data.release_month.value_counts().head(1).index[0]
answers['21']

9

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [39]:
answers['22'] = data.release_month.value_counts().loc[[6,7,8]].sum()
answers['22']

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [40]:
#сначала группируем фильмы с сезоном релиза ЗИМА по "комбинированным" режиссерам 
#это сократит количество строк для дальнейшей обработки 
grouped_data = data[data.release_season == 'winter'].groupby('director')['imdb_id'].count()

#в цикле проходим полученный датафрейм. Считаем количество фильмов по каждому режиссеру отдельно и заносим в словарь
count_movies = {}

for key, value in grouped_data.items():
    
    directors = (key.split('|'))
    
    for director in directors:
        
        if director in count_movies:
            count_movies[director] += value
        else:
            count_movies[director] = value

#превращаем словарь в серию и вычисляем режиссера с максимальным количеством фильмов
answers['23'] = pd.Series(count_movies).sort_values(ascending = False).index[0]
answers['23']

'Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [41]:
#сформируем и заполним словарь с длинами фильмов по каждой студии
title_length = {}

for index, row in data.iterrows():
    
    companies = row['production_companies'].split('|')
    
    for company in companies:
        
        if company in title_length:
            title_length[company].append(len(row['original_title']))
        else:
            title_length[company] = [len(row['original_title'])]    

#сформируем датафрейм из полученного словаря и добавим вычисляемый столбец со средней длиной названий по каждой студии
movies_length_df = pd.DataFrame(pd.Series(title_length),columns = ['movies_length'])
movies_length_df['movies_mean_length'] = movies_length_df.movies_length.apply(lambda x: sum(x)/len(x))

#вычислим студию с максимальной средней длиной фильма
answers['24'] = movies_length_df.sort_values('movies_mean_length',ascending = False).head(1).index[0]
answers['24']

'Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [42]:
#сформируем и заполним словарь с количествами слов в описании фильмов по каждой студии
overview_length = {}

for index, row in data.iterrows():
    
    companies = row['production_companies'].split('|')

    for company in companies:
        
        if company in overview_length:
            overview_length[company].append(len(row['overview'].split(' ')))
        else:
            overview_length[company] = [len(row['overview'].split(' '))]    
        
#сформируем датафрейм из полученного словаря и добавим столбец со средним количеством слов по каждой студии
new_df = pd.DataFrame(pd.Series(overview_length),columns = ['overview_length'])
new_df['overview_mean_length'] = new_df.overview_length.apply(lambda x: sum(x)/len(x))

#вычислим студию с максимальной количеством слов в описании фильма
answers['25'] = new_df.sort_values('overview_mean_length',ascending = False).head(1).index[0]
answers['25']

'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 


In [43]:
#вычисляем количество фильмов, составляющих 1% датафрейма
quantity = len(data)*0.01

#сортируем по рейтингу и выбираем первые фильмы, составляющие 1%
result = data.sort_values('vote_average', ascending = False).head(int(quantity)+1)['original_title']

answers['26'] = result.sort_values().tolist()
answers['26']

['12 Years a Slave',
 'Big Hero 6',
 'Gone Girl',
 'Guardians of the Galaxy',
 'Her',
 'Inception',
 'Inside Out',
 'Interstellar',
 'Memento',
 'Room',
 'Spotlight',
 'The Dark Knight',
 'The Fault in Our Stars',
 'The Grand Budapest Hotel',
 'The Imitation Game',
 'The Lord of the Rings: The Return of the King',
 'The Lord of the Rings: The Two Towers',
 'The Pianist',
 'The Wolf of Wall Street']

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [44]:
count_movies = {}

#пройдемся циклом по каждому фильму
for actors in data.cast:

#вычисляем всевозможные комбинации пар актеров по фильму
    for actor in itertools.combinations(actors.split('|'), 2):
        
#добавляем в словарь пару актеров и увеличиваем счетчик с количеством фильмов      
        if actor in count_movies:
            count_movies[actor] += 1
        else:
            count_movies[actor] = 1

#превратим словарь в серию и вычислим ключи с максимальными значениями
result_df = pd.Series(count_movies)
answers['27'] = result_df[result_df == result_df.max()].keys().tolist()
answers['27']

[('Daniel Radcliffe', 'Rupert Grint'), ('Daniel Radcliffe', 'Emma Watson')]

# Submission

In [45]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals',
 '3': 'Winnie the Pooh',
 '4': 109.6585494970884,
 '5': 107.0,
 '6': ['Avatar', 'tt0499549'],
 '7': ['The Lone Ranger', 'tt1210819'],
 '8': 1478,
 '9': ['The Dark Knight', 'tt0468569'],
 '10': ['The Lone Ranger', 'tt1210819'],
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': ['K-19: The Widowmaker', 'tt0267626'],
 '19': 2015,
 '20': 2014,
 '21': 9,
 '22': 450,
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': ['12 Years a Slave',
  'Big Hero 6',
  'Gone Girl',
  'Guardians of the Galaxy',
  'Her',
  'Inception',
  'Inside Out',
  'Interstellar',
  'Memento',
  'Room',
  'Spotlight',
  'The Dark Knight',
  'The Fault in Our Stars',
  'The Grand Budapest Hotel',
  'The Imitation Game',
  'The Lord of the Rings: The Return of the King',
  'The Lord 

In [46]:
# и убедиться что ни чего не пропустил)
len(answers)

27